In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
raw_reference = pd.read_csv(
	'./scats_reference.csv',
	dtype={
		'Site_Number': np.int32,
		'Location_Descriptors': str,
		'Site_Type': str
	}
)
raw_reference = raw_reference[raw_reference.Site_Type == 'INT']
raw_reference = raw_reference.rename(columns={'Site_Number': 'SCATS_Number'})
raw_reference = raw_reference.drop_duplicates()
raw_reference

,SCATS_Number,Location_Description,Site_Type
0,964,ABBOTTS/CLELANDS DEVELOPMENTS,INT
1,968,ABBOTTS/GAINE/MONASH,INT
2,972,ABBOTTS/NATIONAL,INT
3,983,ABBOTTS/REMINGTON,INT
4,1053,ABBOTTSFORD/HAINES,INT
...,...,...,...
4506,6075,WYNDHAM ST/VAUGHAN,INT
4507,4107,YAN YEAN/IRONBARK,INT
4509,5048,YARRA/EASTERN BEACH,INT
4510,5081,YARRA/LT MALOP,INT


In [3]:
raw_data = pd.read_csv(
	'./scats_data.csv',
	dtype={
		'SCATS_number': np.int32,
		'Location': str,
		'NB_LATITUDE': np.float32,
		'NB_LONGITUDE': np.float32
	}
)
raw_data = raw_data.drop_duplicates()
raw_data

,SCATS_Number,Location,NB_LATITUDE,NB_LONGITUDE
0,970,WARRIGAL_RD N of HIGH STREET_RD,-37.867031,145.091583
31,970,HIGH STREET_RD E of WARRIGAL_RD,-37.867352,145.091949
62,970,WARRIGAL_RD S of HIGH STREET_RD,-37.867599,145.091461
93,970,HIGH STREET_RD W of WARRIGAL_RD,-37.867229,145.091034
123,2000,WARRIGAL_RD N of TOORAK_RD,-37.851685,145.094345
...,...,...,...,...
4042,4812,SWAN_ST SW of MADDEN_GV,-37.829029,145.014557
4068,4821,WALMER_ST N OF VICTORIA_ST,-37.812851,145.008484
4099,4821,VICTORIA_ST E OF BURNLEY_ST,-37.812931,145.008652
4130,4821,BURNLEY_ST S OF VICTORIA_ST,-37.813122,145.008438


In [4]:
ref_df = raw_reference.drop(columns={'Site_Type'})
data_df = raw_data.drop(columns={'Location'})
# Perform an inner merge to keep only SCATS numbers present in both tables
merged_df = pd.merge(ref_df, data_df, on='SCATS_Number', how='inner')
merged_df.to_csv('./merged_data.csv')
merged_df

,SCATS_Number,Location_Description,NB_LATITUDE,NB_LONGITUDE
0,4057,BALWYN/BELMORE,-37.804310,145.081970
1,4057,BALWYN/BELMORE,-37.805080,145.082458
2,4057,BALWYN/BELMORE,-37.805641,145.081711
3,4057,BALWYN/BELMORE,-37.804871,145.080917
4,3001,BARKERS/CHURCH/HIGH,-37.814411,145.022430
...,...,...,...,...
135,3682,WARRIGAL/RIVERSDALE,-37.837410,145.096252
136,4063,WHITEHORSE/BALWYN,-37.814041,145.080093
137,4063,WHITEHORSE/BALWYN,-37.814400,145.080444
138,4063,WHITEHORSE/BALWYN,-37.814758,145.079956


In [5]:
def create_graph(_df: pd.DataFrame):
	'''Take the information from a dataframe and create a graph from it.'''

	locations: dict[int, tuple[float, float]] = {}
	street_to_nodes: dict[str, list[int]] = {}

	for _, row in _df.iterrows():
		scats_num: int = row['SCATS_Number']
		latitude: float = row['NB_LATITUDE']
		longitude: float = row['NB_LONGITUDE']
		loc_desc: str = row['Location_Description']

		# Locations is easy to set up
		locations[scats_num] = (latitude, longitude)

		# Split the location description by '/' to get individual streets
		# Clean and process each street name
		streets = [street.strip() for street in loc_desc.split('/')]

		# Associate each street with the SCATS number
		for street in streets:
			if street:
				if street not in street_to_nodes:
					street_to_nodes[street] = []
				street_to_nodes[street].append(scats_num)

	edge_dict = defaultdict(lambda: defaultdict(int))  # Nested defaultdict for {node: {connected_node: cost}}

	# Connect a node to all other nodes with the same street
	for _, nodes in street_to_nodes.items():
		for node in nodes:
			# Add all other nodes from this street as edges with default cost 1
			for connected_node in nodes:
				if connected_node != node:
					edge_dict[node][connected_node] = 1

	# Convert to a regular dictionary
	edges: dict[int, dict[int, int]] = {node: dict(connected_nodes) for node, connected_nodes in edge_dict.items()}

	return locations, edges

locations, edges = create_graph(merged_df)
print(locations)
print(edges)

{4057: (-37.80487060546875, 145.08091735839844), 3001: (-37.814571380615234, 145.0216064453125), 3002: (-37.815101623535156, 145.0260772705078), 4035: (-37.81880187988281, 145.05738830566406), 4032: (-37.802249908447266, 145.06080627441406), 4040: (-37.83253860473633, 145.05506896972656), 3120: (-37.82284164428711, 145.0568389892578), 4034: (-37.81184005737305, 145.0590057373047), 4030: (-37.794708251953125, 145.0612030029297), 4043: (-37.84714126586914, 145.0520477294922), 2000: (-37.851871490478516, 145.0940704345703), 4266: (-37.82518005371094, 145.0430145263672), 4262: (-37.82154846191406, 145.01502990722656), 4264: (-37.82405090332031, 145.0334930419922), 4263: (-37.82294845581055, 145.02401733398438), 3812: (-37.837249755859375, 145.060546875), 3127: (-37.82527160644531, 145.07757568359375), 3122: (-37.82373809814453, 145.0641632080078), 3126: (-37.827720642089844, 145.09829711914062), 2820: (-37.7947998046875, 145.0301513671875), 4324: (-37.80917739868164, 145.0364532470703), 31

In [6]:
# Print basic graph info
print(f'Number of nodes (intersections): {len(locations)}')
print(f'Number of edges (street connections): {len(edges)}')

# List all nodes and their attributes
for node, (latitude, longitude) in locations.items():
	print(f'{node:4}: ({latitude:.6f}, {longitude:.6f})')

# List all edges and their cost
for node, others in edges.items():
	for other, cost in others.items():
		print(f'{node:4} -- {other:4}: Cost = {cost}')

Number of nodes (intersections): 40
Number of edges (street connections): 35
4057: (-37.804871, 145.080917)
3001: (-37.814571, 145.021606)
3002: (-37.815102, 145.026077)
4035: (-37.818802, 145.057388)
4032: (-37.802250, 145.060806)
4040: (-37.832539, 145.055069)
3120: (-37.822842, 145.056839)
4034: (-37.811840, 145.059006)
4030: (-37.794708, 145.061203)
4043: (-37.847141, 145.052048)
2000: (-37.851871, 145.094070)
4266: (-37.825180, 145.043015)
4262: (-37.821548, 145.015030)
4264: (-37.824051, 145.033493)
4263: (-37.822948, 145.024017)
3812: (-37.837250, 145.060547)
3127: (-37.825272, 145.077576)
3122: (-37.823738, 145.064163)
3126: (-37.827721, 145.098297)
2820: (-37.794800, 145.030151)
4324: (-37.809177, 145.036453)
3180: (-37.796181, 145.083282)
4051: (-37.794060, 145.068848)
2827: (-37.781269, 145.076874)
2825: (-37.786610, 145.062027)
4270: (-37.830151, 145.032074)
4335: (-37.806190, 145.035324)
3662: (-37.808922, 145.027039)
4321: (-37.800629, 145.048645)
2200: (-37.816479, 145.0

In [7]:
import search

method = search.select_method('AS')

if method is None:
	print("Incorrect method type, valid methods:\nDFS, BFS, GBFS, AS, CUS1, CUS2, IDS, BS")
	quit()

graph = search.Graph(edges)
graph.locations = locations

origin = 4030
goals = [4035]

problem = search.GraphProblem(origin, goals, graph)

result, count = method(problem, False)

print('method=AS')
# \n
# Ouput goal node
print('goal=', goals, sep='', end=' | ')

# Output number (length of path)
print('number of nodes=', count, sep='')
# \n
if (result is not None):
	# Output path: list of nodes
	print('path=', result.solution(), sep='')
else:
	print('No path found!')

method=AS
goal=[4035] | number of nodes=35
No path found!
